In [67]:
import pandas as pd

In [231]:
article_results = {
    'Article3_full': 0.70,
    'Article3_procedure': 0.67,
    'Article3_circumstances': 0.68,
    'Article3_relevantLaw': 0.68,
    'Article3_circumstances_relevantLaw': 0.70,
    'Article3_law': 0.56,
    'Article3_topics': 0.78,
    'Article3_topics_circumstances': 0.75,
    'Article6_full': 0.82,
    'Article6_procedure': 0.81,
    'Article6_circumstances': 0.82,
    'Article6_relevantLaw': 0.78,
    'Article6_circumstances_relevantLaw': 0.78,
    'Article6_law': 0.68,
    'Article6_topics': 0.81,
    'Article6_topics_circumstances': 0.84,
    'Article8_full': 0.72,
    'Article8_procedure': 0.71,
    'Article8_circumstances': 0.77,
    'Article8_relevantLaw': 0.72,
    'Article8_circumstances_relevantLaw': 0.68,
    'Article8_law': 0.62,
    'Article8_topics': 0.76,
    'Article8_topics_circumstances': 0.78,
    'average_full': 0.75,
    'average_procedure': 0.73,
    'average_circumstances': 0.76,
    'average_relevantLaw': 0.73,
    'average_circumstances_relevantLaw': 0.73,
    'average_law': 0.62,
    'average_topics': 0.78,
    'average_topics_circumstances': 0.79
}

In [68]:
DATA = 'ech_dataset'

In [69]:
articles = !ls echr_dataset

In [165]:
!ls echr_dataset/Article8/

cases_a8.csv		    ngrams_a8_full.csv	     ngrams_a8_relevantLaw.csv
ngrams_a8_circumstanes.csv  ngrams_a8_laws.csv	     topics8.csv
ngrams_a8_featureNames.txt  ngrams_a8_procedure.csv  topics8_vocab.txt


In [166]:
def get_article(path, features):
    files_raw = !ls {path}
    files_features = ['cases', 'circumstances', 'featureNames', 'full', 'law', 'procedure', 'relevantLaw', 'topics', 'topicsVocab']
    files = dict(zip(files_features, files_raw))
    
    group = []
    for feature in features:
        if(feature == 'topics'): raw = pd.read_csv(path+files[feature], sep='\t', header=None)
        elif(feature == 'topicsVocab'): raw = open(path+files[feature]).read(); raw.replace('\n', '').replace(', ', ',').split(',')
        else: raw = pd.read_csv(path+files[feature], header=None)
        
        if feature == 'cases': raw = raw[1]
            
        group.append(raw)
    return pd.concat(group, axis=1, ignore_index=True)

In [167]:
!ls echr_dataset

Article3  Article6  Article8


In [257]:
data = get_article('echr_dataset/Article8/', ['cases', 'full'])

In [258]:
X = data.iloc[:, 1:]

In [259]:
y = data[0]
y = np.array([1 if i == 'v' else 0  for i in y])

In [260]:
X[:5], y[:5]

(       1     2     3         4         5     6     7     8         9     \
 0  0.004009   0.0   0.0  0.000000  0.001145   0.0   0.0   0.0  0.000573   
 1  0.000156   0.0   0.0  0.000156  0.000781   0.0   0.0   0.0  0.000000   
 2  0.000000   0.0   0.0  0.000354  0.000000   0.0   0.0   0.0  0.001772   
 3  0.000000   0.0   0.0  0.000000  0.000000   0.0   0.0   0.0  0.000000   
 4  0.000000   0.0   0.0  0.000000  0.000000   0.0   0.0   0.0  0.000000   
 
        10    ...       1991      1992      1993  1994      1995      1996  \
 0  0.004582  ...   0.000000  0.000000  0.000000   0.0  0.000573  0.002291   
 1  0.001093  ...   0.010303  0.000312  0.000312   0.0  0.000000  0.000468   
 2  0.000709  ...   0.000354  0.000000  0.000354   0.0  0.000000  0.002481   
 3  0.005333  ...   0.000000  0.000000  0.000000   0.0  0.000000  0.003556   
 4  0.000000  ...   0.000000  0.000000  0.000000   0.0  0.000000  0.000000   
 
        1997      1998  1999  2000  
 0  0.000000  0.001145   0.0   0.0 

In [261]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

import numpy as np

In [173]:
from utils.nbsvm import NBSVM

In [200]:
from termcolor import colored

In [144]:
param_grid = [
    {'C': [1, 5, 100, 1000], 'alpha': [0.0001]}
]

In [185]:
clf = NBSVM(C=1000, alpha=0.00001, beta=0.25)

In [186]:
feature_tests = [
    ['cases', 'full'],
    ['cases', 'procedure'],
    ['cases', 'circumstances'],
    ['cases', 'relevantLaw'],
    ['cases', 'circumstances', 'relevantLaw'],
    ['cases', 'law'],
    ['cases', 'topics'],
    ['cases', 'topics', 'circumstances']
]

In [197]:
classifiers = {
    'SVM Linear': SVC(C=1),
    'Naive Bayes SVM': NBSVM(C=1000, alpha=0.00001, beta=0.25)
}

In [232]:
articles = !ls echr_dataset

for classifier_name, classifier in classifiers.items():
    print(classifier_name)
    for feature_test in feature_tests:
        feature_mean = []
        for article in articles:
            data = get_article(f'echr_dataset/{article}/', feature_test)
            X = data.iloc[:, 1:]
            y = data[0]
            y = np.array([1 if i == 'v' else 0  for i in y])
            score = np.mean(cross_val_score(classifier, X, y, cv=10))
            feature_mean.append(score)
            if score >= article_results['{}{}'.format(article, ''.join(['_'+feature for feature in feature_test[1:]]))]: c = 'green'
            else: c = 'red'
            print(colored(f'{article} - {feature_test}: {score}', c))
        if np.mean(feature_mean) >= article_results['average{}'.format(''.join(['_'+feature for feature in feature_test[1:]]))]: c = 'green'
        else: c = 'red'
        print(colored(f'{feature_test} - mean: {np.mean(feature_mean)}', c))
    print('-'*40)

SVM Linear
Article3 - ['cases', 'full']: 0.6858974358974359
Article6 - ['cases', 'full']: 0.825
Article8 - ['cases', 'full']: 0.6605769230769231
['cases', 'full'] - mean: 0.7238247863247863
Article3 - ['cases', 'procedure']: 0.6503205128205127
Article6 - ['cases', 'procedure']: 0.7
Article8 - ['cases', 'procedure']: 0.525
['cases', 'procedure'] - mean: 0.6251068376068375
Article3 - ['cases', 'circumstances']: 0.6775641025641026
Article6 - ['cases', 'circumstances']: 0.825
Article8 - ['cases', 'circumstances']: 0.7073717948717949
['cases', 'circumstances'] - mean: 0.7366452991452991
Article3 - ['cases', 'relevantLaw']: 0.5237179487179487
Article6 - ['cases', 'relevantLaw']: 0.6125
Article8 - ['cases', 'relevantLaw']: 0.5673076923076923
['cases', 'relevantLaw'] - mean: 0.5678418803418804
Article3 - ['cases', 'circumstances', 'relevantLaw']: 0.5358974358974359
Article6 - ['cases', 'circumstances', 'relevantLaw']: 0.6125
Article8 - ['cases', 'circumstances', 'relevantLaw']: 0.6266025641025